# 1. Import Libraris

In [1]:
import cv2
import numpy as np
import os

from matplotlib import pyplot as plt
import time
!pip install mediapipe
import mediapipe as mp
import shutil
from tqdm import tqdm

In [6]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install unrar

Note: you may need to restart the kernel to use updated packages.


# **Skip To section 4 to download the pre-processed data that's already done using sections 2, 3**

#2. Convert videos (dataset) into keppoints frames

In [ ]:
#Download Dataset. SKIP if exist 
!gdown --id 1V6cozzvTo6vfN5hWaSiQF0gfkXcYYmIx
!unrar x dataset_v2.rar -o+

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
os.mkdir('MP_Data')
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array([name for name in os.listdir("dataset")])
actions = np.sort(actions)

# Number of frames per any single video
sequence_length = 40

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'dataset'

In [ ]:
#create folders to hold dataset of np arrays; subfolder for each class/word
for action in actions: 
    os.mkdir('MP_Data/'+action)
    
    file_names = os.listdir("dataset/"+action)
    file_names = [int(x[:-4]) for x in file_names]
    num_of_vids = max(file_names)
    for i in range(0,num_of_vids+1):
        os.mkdir('MP_Data/'+action+"/"+str(i))

In [ ]:
# Create detections of pose and hands in npy format

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh, rh])

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for action in tqdm(actions):
        
        file_names = os.listdir("dataset/"+action)
        file_names = [int(x[:-4]) for x in file_names]
        num_of_vids = max(file_names)
        
        # Loop through sequences aka videos
        for sequence in range(0, num_of_vids+1):
            
            vidcap = cv2.VideoCapture('dataset/{}/{}.mp4'.format(action,sequence))
            success,frame = vidcap.read()
            count = -1
            # Loop through each frame
            while success:  
                count += 1

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(count))
                np.save(npy_path, keypoints)
                success,frame = vidcap.read()

100%|███████████████████████████████████████████████████████████████████████████████| 52/52 [3:43:22<00:00, 257.75s/it]


#3. Filter videos to select 20 frame out of 40
using action detection algorithm

In [ ]:
#SETUP FOLDERS FOR FILTERED DATA
os.mkdir('MP_Data_FILTERED')
DATA_PATH = os.path.join('MP_Data_FILTERED') 
actions = np.array([name for name in os.listdir("dataset")])
actions = np.sort(actions)
sequence_length = 20
number_of_frames = 20

In [ ]:
#create folders to hold dataset of np arrays
for action in actions: 
    os.mkdir('MP_Data_FILTERED/'+action)
    file_names = os.listdir("dataset/"+action)
    file_names = [int(x[:-4]) for x in file_names]
    num_of_vids = max(file_names)
    for i in range(0,num_of_vids+1):
        os.mkdir('MP_Data_FILTERED/'+action+"/"+str(i))

In [ ]:
# DO THE PROCESS

# Loop through actions (words)
for action in tqdm(actions):
    file_names = os.listdir("dataset/"+action)
    file_names = [int(x[:-4]) for x in file_names]
    num_of_vids = max(file_names)
    # Loop through sequences (videos)
    for sequence in range(0, num_of_vids+1):
        vidcap = cv2.VideoCapture('dataset/{}/{}.mp4'.format(action,sequence))
        success,frameTemp = vidcap.read() #FRAME IS IMAGE
        success,frame = vidcap.read() 
        count = -1
        actionMagnitude = []
        # Loop through each frame in a single video, to find the action magnitude over each frame in a video
        while success:
            count += 1
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            gray = cv2.GaussianBlur(gray, (21, 21), 0)
            static_back = cv2.cvtColor(frameTemp, cv2.COLOR_BGR2GRAY)
            static_back = cv2.GaussianBlur(static_back, (21, 21), 0)
            diff_frame = cv2.absdiff(static_back, gray)
            actionMagnitude.append(diff_frame.sum())
            frameTemp = np.copy(frame)
            success,frame = vidcap.read() #FRAME IS IMAGE

        #find best n frames to extract************************************
        maxIndex = 0 #best index to start collecting max(n = 20) frame. ex: max=5 means best is to collect frames[5:25]
        maxMagnitude = 0 #for a given number_of_frames
        for i in range(0,len(actionMagnitude)-number_of_frames):
            if sum(actionMagnitude[i:i+number_of_frames]) > maxMagnitude:
                maxIndex = i
                maxMagnitude = sum(actionMagnitude[i:i+number_of_frames])

        #Copy selected frames' keypoints into another location************
        for i in range(maxIndex,maxIndex+number_of_frames):
            source = "MP_Data/{}/{}/{}.npy".format(action,sequence,i)
            destination = "MP_Data_FILTERED/{}/{}/{}.npy".format(action,sequence,i-maxIndex) #renamed to be all starting from 0
            shutil.copyfile(source, destination)

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [28:21<00:00, 32.72s/it]


# 4. Download pre-processed keypoints data 

In [8]:
import gdown

In [9]:
import unrar

In [10]:
#download keypoints (MP_Data_FILTERED_v3.rar)
!gdown --id 10vZEV7lnshZEFzUp7ZTZuQ4ZiUmnbePY
!unrar x MP_Data_FILTERED_v3.rar -o+

C:\Users\mai\anaconda3\lib\site-packages\gdown\__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=10vZEV7lnshZEFzUp7ZTZuQ4ZiUmnbePY
From (redirected): https://drive.google.com/uc?id=10vZEV7lnshZEFzUp7ZTZuQ4ZiUmnbePY&confirm=t&uuid=b51858dc-2f19-4364-9702-69a79899853b
To: C:\Users\mai\Documents\ss\sign_language_translator\Traning\MP_Data_FILTERED_v3.rar

  0%|          | 0.00/75.9M [00:00<?, ?B/s]
  1%|          | 524k/75.9M [00:00<01:12, 1.05MB/s]
  1%|1         | 1.05M/75.9M [00:00<00:46, 1.62MB/s]
  2%|2         | 1.57M/75.9M [00:00<00:37, 1.97MB/s]
  3%|2         | 2.10M/75.9M [00:01<00:35, 2.09MB/s]
  3%|3         | 2.62M/75.9M [00:01<00:31, 2.35MB/s]
  4%|4         | 3.15M/75.9M [00:01<00:27, 2.62MB/s]
  5%|4         | 3.67M/75.9M [00:01<00:25, 2.80MB/s]
  6%|5         | 4.19M/75.9M [00:01<00:25, 

'unrar' is not recognized as an internal or external command,
operable program or batch file.


# 5. Load data into memory

In [2]:
actions = np.array([name for name in os.listdir("MP_Data_FILTERED")])
actions = np.sort(actions)
sequence_length = 20
label_map = {label:num for num, label in enumerate(actions)}
DATA_PATH = os.path.join('MP_Data_FILTERED') 

In [5]:
res.shape

(258,)

In [3]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
print('Data loaded!\ndata size: ',np.array(sequences).shape)
print('labels size: ',np.array(labels).shape)


KeyboardInterrupt: 

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = np.array(sequences)
y = to_categorical(labels).astype(int)

#DO FLIP HORIZONTAL ON ALL (DATA AUGMENTATION)
X = np.concatenate([X,X*-1])
y = np.concatenate([y,y])
#*********************************************

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42, shuffle=True)

print('Test       : ',y_test.shape)
print('Validation : ',y_val.shape)
print('Train      : ',y_train.shape)

Test       :  (686, 52)
Validation :  (617, 52)
Train      :  (5549, 52)


# 6. Modeling

In [19]:
pip install rm

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rm: filename=rm-2020.12.3-py3-none-any.whl size=1368 sha256=32fe64b9eef293cd28981f132d59be57b8fecd522704b4ececf4ce4bc7c93969
  Stored in directory: c:\users\mai\appdata\local\pip\cache\wheels\65\c1\c7\66fd3f38413f44d9d33e30954acba16d681f3ab48dd512746b
  Created wheel for values: filename=values-2020.12.3-py3-none-any.whl size=1341 sha256=979fd7b526a3919a00913f715ea16aa619d058772d769e58fda7e3ade22305b6
  Stored in directory: c:\users\mai\appdata\local\pip\cache\wheels\8b\47\31\cb52254b8a2df484d2a7e29900de831b43155546c9a72f3456
Successfully built rm values
Note: you may need to restart the kernel to use updated packages.


In [21]:
import rm

In [22]:
!rm -r Logs

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()

model.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh', input_shape=(np.array(sequences).shape[1],np.array(sequences).shape[2]))))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh')))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Bidirectional(LSTM(256, return_sequences=False, activation='tanh')))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam',  loss='categorical_crossentropy', metrics=['categorical_accuracy'])

checkpoint = ModelCheckpoint('weights.hdf5',monitor='val_categorical_accuracy',save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', patience=60)
earlystoppingmonitor =  EarlyStopping(monitor='val_categorical_accuracy',patience=130)

#training = model.fit(X_train, y_train,  batch_size=512, epochs=600,validation_data=(X_val,y_val),
         # callbacks=[checkpoint,earlystoppingmonitor,reduce_lr,tb_callback])

In [15]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [3]:
from tensorflow.keras.models import load_model
model = load_model('final.keras')

In [3]:
model.load_weights('final.h5')

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 20, 256)           396288    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 20, 256)           0         
                                                                 
 batch_normalization (Batch  (None, 20, 256)           1024      
 Normalization)                                                  
                                                                 
 bidirectional_1 (Bidirecti  (None, 20, 256)           394240    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 20, 256)           0         
                                                        

# 7. Testing and Evaluating

In [8]:
from tensorflow.keras.models import load_model
from sklearn.metrics import  accuracy_score

#using the model checkpoint
best = load_model('weights.hdf5')
yhat = best.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
print("Testing Accuracy (Checkpoint): ",accuracy_score(ytrue, yhat))

#using the full trained model (till last executed epoch)
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
print("Testing Accuracy: ",accuracy_score(ytrue, yhat))

NameError: name 'X_test' is not defined

In [19]:
#using the model checkpoint
best = load_model('weights.hdf5')
yhat = best.predict(X_val)
ytrue = np.argmax(y_val, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
print("Val Accuracy (Checkpoint): ",accuracy_score(ytrue, yhat))

yhat = best.predict(X_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
print("Train Accuracy (Checkpoint): ",accuracy_score(ytrue, yhat))

20/20 [==============================] - 3s 71ms/step
Val Accuracy (Checkpoint):  0.940032414910859
174/174 [==============================] - 15s 86ms/step
Train Accuracy (Checkpoint):  1.0


# 8. Save Model

In [28]:
model.save('final.keras')

# 9.Testing in real time

In [5]:
from scipy import stats

In [6]:
colors = [(245,117,16), (117,245,16), (16,117,245),(255, 0, 0),(0, 255, 0),(0, 0, 255),(255, 255, 0)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [7]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image , colors))

NameError: name 'res' is not defined

<Figure size 1800x1800 with 0 Axes>

In [8]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [9]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [10]:
def draw_landmarks(image, results):
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [11]:
def draw_styled_landmarks(image, results):
    # Draw face connections
   # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                         ##    mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                            # mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                            # ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [12]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [26]:
sign_dict = {
    "iam": "أنا",
    "you": "أنت",
    "sorry": "أسف",
    "understand": "يفهم",
    "okay": "تمام",
    "good": "جيد",
    "morning": "صباح",
    "help": "يساعد",
    "nice": "لطيف",
    "meet": "يقابل",
    "yes": "نعم",
    "no": "لا",
    "hello": "مرحبا",
    "excuse": "عذر",
    "love": "حب",
    "egypt": "مصر",
    "hate": "يكره",
    "have": "يملك",
    "car": "سيارة",
    "fun": "هزار",
    "take": "يأخذ",
    "picture": "صورة",
    "holiday": "عطلة",
    "enjoy": "يتمتع",
    "time": "وقت",
    "not": "لا",
    "every": "كل",
    "day": "يوم",
    "my": "لي",
    "your": "لك",
    "when": "متى",
    "where": "أين",
    "what": "ماذا",
    "name": "اسم",
    "age": "عمر",
    "thanks": "شكرا",
    "forget": "ينسى",
    "speak": "يتكلم",
    "english": "إنجليزي",
    "call": "يتصل",
    "number": "رقم",
    "sport": "رياضة",
    "football": "كرة القدم",
    "run": "يجري",
    "book": "كتاب",
    "read": "يقرأ",
    "play": "يلعب",
    "ride": "يركب",
    "today": "اليوم",
    "month": "شهر",
    "year": "سنة",
    "na": "لا شئ"
}

In [28]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5
prediction=0
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        print(keypoints.shape)
        sequence.append(keypoints)
        sequence = sequence[-20:]
        
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            prediction=actions[np.argmax(res)]
            print(sign_dict[actions[np.argmax(res)]])
            predictions.append(np.argmax(res))
            cv2.putText(image, ' '.join(prediction), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
    

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
<class 'mediapipe.python.solution_base.S

1/1 [==============================] - 0s 31ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 45ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 51ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 35ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 46ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 38ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 40ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 57ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)


1/1 [==============================] - 0s 35ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 40ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 33ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 41ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 39ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 33ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 37ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 50ms/step
okay
تمام
<class 'mediapipe.python.solution_base.SolutionOutputs'>

1/1 [==============================] - 0s 34ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 37ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 40ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 35ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 42ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 36ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 39ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 37ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionO

love
حب
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 35ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 39ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 37ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 33ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 43ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 58ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 44ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 39ms/step
not
لا
<class '

...

na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 41ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 39ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 44ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 49ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 35ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 47ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 34ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 36ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 38ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 36ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 40ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 37ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 50ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 79ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 38ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.S

1/1 [==============================] - 0s 40ms/step
na
لا شئ
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 43ms/step
love
حب
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 48ms/step
love
حب
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 49ms/step
love
حب
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 46ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 32ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 39ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 47ms/step
not
لا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [======

1/1 [==============================] - 0s 36ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 40ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 37ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 29ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 31ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 31ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 39ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 59ms/step
thanks
شكرا
<class 'mediapipe.python.solution_base.S

1/1 [==============================] - 0s 37ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 32ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 52ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 42ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 49ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 49ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 65ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionOutputs'>
(258,)
1/1 [==============================] - 0s 43ms/step
help
يساعد
<class 'mediapipe.python.solution_base.SolutionO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/final.h5 /content/drive/MyDrive/SIGNXv3.3_94.8/final.h5
!cp /content/weights.hdf5 /content/drive/MyDrive/SIGNXv3.3_94.8/weights.hdf5